In [9]:
import numpy as np
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms, Compose
from torchvision import datasets,utils
from torch import nn
import matplotlib.pyplot as plt
import os


learning_rate = 1e-3
batch_size = 64
epochs = 10 
accuracies = []
losses = []
epochs = []
def get_device_enabled():
    return torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"



class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32*8*8,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
        self.conv_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
    def forward(self,x):
        x = self.conv_stack(x)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = ConvolutionalNeuralNetwork()
print(model)    
torch.manual_seed(63)     

ConvolutionalNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (conv_stack): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


In [10]:

def load_data(data_transforms:Compose):

    mean = [0.47889522, 0.47227842, 0.43047404]
    std = [0.24205776, 0.23828046, 0.25874835]
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    training_data = datasets.ImageFolder(root="./cinic-10/train", transform=transform)
    test_data = datasets.ImageFolder(root="./cinic-10/test", transform=transform)
    validation_data = datasets.ImageFolder(root="./cinic-10/valid", transform=transform)


    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64)

    return train_dataloader, test_dataloader


# def sample_data_raw():
#     root_directory = os.getcwd()
#     train_directory = os.path.join(root_directory,'train_dataset')
#     category_list = os.listdir(train_directory)
#     for category in category_list:
#         for i in range(5)

def display_image(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


def sample_data(dataloader:DataLoader):
    data_iterable = iter(dataloader)
    images,label = next(data_iterable)
    display_image(torchvision.utils.make_grid(images))



#step by step
#set model into train mode
#get x,y, values
#predict
#calculate loss


def train(dataloader:DataLoader, model:ConvolutionalNeuralNetwork, loss_fn,optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch,(X,Y) in enumerate(dataloader):
        x,y = X,Y
        prediction = model(x)
        loss = loss_fn(prediction,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader:DataLoader,model:ConvolutionalNeuralNetwork,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,Y in dataloader:
            x,y = X,Y
            prediction = model(x)
            loss = loss_fn(prediction,y)
            test_loss += loss
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")







In [11]:
if __name__ == "__main__":
    data_transforms = transforms.Compose([
        transforms.ToTensor(),
    ])
    training_dataloader, test_dataloader = load_data(data_transforms)
    model = ConvolutionalNeuralNetwork().to()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    

In [ ]:
epochs = 5
for t in range(epochs):
        print(f"Epochs------------{t+1}")
        print("__________________________")
        train(training_dataloader,model,loss_fn, optimizer)
        test(test_dataloader,model,loss_fn)

torch.save(model.state_dict(),'./my_models/saved_model_conv.pth')

Epochs------------1
__________________________
loss: 2.293132  [   64/90000]
loss: 1.842283  [ 6464/90000]
loss: 1.484269  [12864/90000]
loss: 1.466781  [19264/90000]
loss: 1.607414  [25664/90000]
loss: 1.469501  [32064/90000]
loss: 1.535515  [38464/90000]
loss: 1.397517  [44864/90000]
loss: 1.235913  [51264/90000]
loss: 1.370139  [57664/90000]
loss: 1.131430  [64064/90000]
loss: 1.267992  [70464/90000]
loss: 1.418838  [76864/90000]
loss: 1.153772  [83264/90000]
loss: 1.261406  [89664/90000]
Test Error: 
 Accuracy: 53.3%, Avg loss: 1.270709 

Epochs------------2
__________________________
loss: 1.214388  [   64/90000]
loss: 1.225283  [ 6464/90000]
loss: 1.079461  [12864/90000]
loss: 1.187008  [19264/90000]
loss: 1.111267  [25664/90000]
loss: 1.000626  [32064/90000]
loss: 1.059279  [38464/90000]
loss: 1.558179  [44864/90000]
loss: 0.958698  [51264/90000]
loss: 1.349134  [57664/90000]
loss: 1.374758  [64064/90000]
loss: 1.150668  [70464/90000]
loss: 0.925319  [76864/90000]
loss: 1.409567